In [12]:
from __future__ import print_function
from random import random
from numpy import array
from numpy import cumsum
from keras.optimizers import SGD, Adam
from keras.models import Sequential
from keras.layers import LSTM, Embedding
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Bidirectional, Concatenate, concatenate, Dropout, Input, Merge, TimeDistributed, Flatten
from keras.layers import Convolution1D, MaxPooling1D, GlobalMaxPooling1D
from keras.models import Model
import data_set
from data_set import load_embeddings, fetch_data, fetch_embeddings
from data_set import MAX_SEQUENCE_LENGTH_HEAD
from data_set import MAX_SEQUENCE_LENGTH_BODY
from data_set import EMBEDDING_DIM
from data_set import VALIDATION_SPLIT
from data_set import MAX_NUM_WORDS, embeddings_index, labels_index
import numpy as np
from keras.utils import to_categorical
import pickle

In [13]:
load_embeddings(BASE_DIR = '/Users/avinashaita/Desktop/DeepLearning/Keras Tutorial')
(headlines, bodies_map, labels) = fetch_data(BASE_DIR = '/Users/avinashaita/Desktop/DeepLearning/Keras Tutorial/data')

Found 400000 word vectors.
All done


In [7]:
(data_head, data_body, embedding_layer_head, embedding_layer_body) = fetch_embeddings(headlines, bodies_map)

Preparing embedding matrix.
All done


In [8]:
pickle.dump((headlines, bodies_map, labels), open("train_file1.txt", "wb"))
pickle.dump((data_head, data_body, embedding_layer_head, embedding_layer_body), open("train_file2.txt", "wb"))

In [9]:
(headlines, bodies_map, _) = pickle.load(open("train_file1.txt", "rb"))
(data_head, data_body, embedding_layer_head, embedding_layer_body) = pickle.load(open("train_file2.txt", "rb"))

In [14]:
#Define model architecture

#Declare Layers
n_classes = 4
emb_size = EMBEDDING_DIM
n_hidden = 32
width = 5

#Conv
conv_0  = Convolution1D(filters = n_hidden, kernel_size = width, activation='relu', padding='same')
conv_1  = Convolution1D(filters = n_hidden, kernel_size = width, activation='relu', padding='same')
conv_2  = Convolution1D(filters = 2*n_hidden, kernel_size = width, activation='relu', padding='same')
conv_3  = Convolution1D(filters = 2*n_hidden, kernel_size = width, activation='relu', padding='same')
conv_4  = Convolution1D(filters = 3*n_hidden, kernel_size = width, activation='relu', padding='same')

#Dense
dense_0 = Dense(n_hidden*4, activation='relu')
dense_1 = Dense(n_hidden*4, activation='relu')
dense_2 = Dense(n_hidden*4, activation='relu')
dense_f = Dense(n_classes, activation='softmax', name='out')

#Drop
drop_0 = Dropout(0.25)
drop_1 = Dropout(0.25)
drop_2 = Dropout(0.25)
drop_3 = Dropout(0.25)
drop_4 = Dropout(0.25)

#Pooling
pool_0 = MaxPooling1D(pool_size=3, padding='same')
pool_1 = MaxPooling1D(pool_size=3, padding='same')
pool_2 = MaxPooling1D(pool_size=3, padding='same')

#Input formats
input_title   = Input(shape=(MAX_SEQUENCE_LENGTH_HEAD, ), dtype='int32', name='input_title')
input_body   = Input(shape=(MAX_SEQUENCE_LENGTH_BODY, ), dtype='int32', name='input_body')

#Create Layers - Title
x = input_title
x = embedding_layer_head(x)
#x = dim_shuffle(x)
x = conv_0(x)
x = drop_0(x)
x = pool_0(x)
#x = conv_1(x)
#x = drop_1(x)
#x = pool_1(x)
#x = conv_2(x)
#x = drop_2(x)
#x = pool_2(x)
x = conv_3(x)
x = drop_3(x)
x = conv_4(x)
x = drop_4(x)

#Create Layers - Body
y = input_body
y = embedding_layer_body(y)
#y = dim_shuffle(y)
y = conv_0(y)
y = drop_0(y)
y = pool_0(y)
#y = conv_1(y)
#y = drop_1(y)
#y = pool_1(y)
#y = conv_2(y)
#y = drop_2(y)
#y = pool_2(y)
y = conv_3(y)
y = drop_3(y)
y = conv_4(y)
y = drop_4(y)

#Max selection
x = GlobalMaxPooling1D()(x)
y = GlobalMaxPooling1D()(y)

#Concatenate Layers
z = concatenate([x, y])
#z = Merge(mode="concat",name="concat_layer")([x_new, y_new])

#Create Dense layers
z = dense_0(z)
#z = dense_1(z)
#z = dense_2(z)

#Output layer
out = dense_f(z)

#Summary
model = Model(inputs=[input_title, input_body], outputs=[out])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_title (InputLayer)        (None, 50)           0                                            
__________________________________________________________________________________________________
input_body (InputLayer)         (None, 200)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 50, 50)       194000      input_title[0][0]                
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 200, 50)      1000000     input_body[0][0]                 
__________________________________________________________________________________________________
conv1d_1 (

In [15]:
#Compile
optimizer = Adam(lr=0.0002, beta_1=0.1, beta_2=0.001, epsilon=1e-08) # Optimization hyperparameters.

model.compile(optimizer=optimizer,
              loss={'out':'categorical_crossentropy'},
              loss_weights={'out': 1.0}, # These can be tuned.
              metrics=['accuracy'])
#lr=0.0002, b1=0.1, b2=0.001, e=1e-8

In [17]:
#Train and Validate
labels = to_categorical(np.asarray(list(zip(*headlines))[2]))
model.fit([data_head[20000:25000], data_body[20000:25000]], labels[20000:25000],
          validation_data=([data_head[1000:2000], data_body[1000:2000]], labels[1000:2000]),
          batch_size=24, epochs=5)

Train on 5000 samples, validate on 1000 samples
Epoch 1/5
5000/5000 [==============================] - 13s 3ms/step - loss: 0.8499 - acc: 0.7184 - val_loss: 0.8996 - val_acc: 0.7630
Epoch 2/5
5000/5000 [==============================] - 14s 3ms/step - loss: 0.8307 - acc: 0.7194 - val_loss: 0.8526 - val_acc: 0.7630
Epoch 3/5
5000/5000 [==============================] - 14s 3ms/step - loss: 0.8140 - acc: 0.7212 - val_loss: 0.8899 - val_acc: 0.7640
Epoch 4/5
5000/5000 [==============================] - 13s 3ms/step - loss: 0.8027 - acc: 0.7226 - val_loss: 0.8792 - val_acc: 0.7660
Epoch 5/5
5000/5000 [==============================] - 13s 3ms/step - loss: 0.7971 - acc: 0.7232 - val_loss: 0.8165 - val_acc: 0.7660


In [18]:
#Confusion matrix

# Predicting the Test set results
y_pred = model.predict([data_head[20000:25000], data_body[20000:25000]])

lab = np.argmax(labels[20000:25000], axis = 1)
pred = np.argmax(y_pred, axis = 1)

from sklearn.metrics import confusion_matrix
confusion_matrix(lab, pred)

array([[3604,    0,    0,    3],
       [ 375,    0,    0,    1],
       [  99,    0,    0,    0],
       [ 898,    0,    0,   20]])